In [4]:
import io
from IPython.display import clear_output
import regex
from opencc import OpenCC
from gensim.models import KeyedVectors
import math
from os import listdir
from os.path import isfile, isdir, join
import os
import requests

In [38]:
#festtest_chinese讀檔
def load_vectors(fname):
    global AL_CBOW_Data
    print('初始化中')
    AL_CBOW_Data = KeyedVectors.load_word2vec_format('cc.zh.300.vec')
    clear_output(wait=True)
    print('完成')
    return


In [39]:
#cosine距離運算
def cosine_meansure(word1,word2):
    word1 = OpenCC('s2t').convert(word1)
    word2 = OpenCC('s2t').convert(word2)
    #判斷此兩個詞是否有出現過
    if (not word1 in AL_CBOW_Data) :
        print('We do not have ['+word1+']')
    if (not word2 in AL_CBOW_Data) :
        print('We do not have ['+word2+']')
    if(not word1 in AL_CBOW_Data) or (not word2 in AL_CBOW_Data) :
        #print('出現此訊息，代表因為我們產生Word2Vec的資料庫沒有這個詞彙，所以無法計算!')
        return 0.0
    #開始計算餘弦相關性
    molecule_A = 0.0 #詞彙1的分母計算
    molecule_B = 0.0 #詞彙2的分母計算
    Denominator = 0.0 #交互的分子計算
    for index in range(len(AL_CBOW_Data[word1])):
        Denominator += AL_CBOW_Data[word1][index] * AL_CBOW_Data[word2][index]
        molecule_A += AL_CBOW_Data[word1][index] * AL_CBOW_Data[word1][index]
        molecule_B += AL_CBOW_Data[word2][index] * AL_CBOW_Data[word2][index]
    molecule_all = math.sqrt(molecule_A)*math.sqrt(molecule_B)
    return (Denominator / molecule_all)


In [40]:
#先把題組間的距離算好
def ready_for_distance(dictionary_a2p,key):#dictionary_a2p is list;key = the answer
    data_a2p = []
    data_p2p = [[0 for i in range(len(dictionary_a2p))] for j in range(len(dictionary_a2p))]
    
    count = 0
    print('計算' + key)
    for value in dictionary_a2p:
        data_a2p.append (cosine_meansure(key,value))
        #print('第'+str(count)+'數'+value+'與a：'+str(data_a2p[count]))
        count += 1
    
    for i in range(len(dictionary_a2p)):
        clear_output(wait=True)
        print('計算' +'第'+str(i)+'數'+ dictionary_a2p[i])
        for j in range(i+1,len(dictionary_a2p)):
            data_p2p[i][j] = cosine_meansure(dictionary_a2p[i],dictionary_a2p[j])
            #print('第'+str(i)+'數'+dictionary_a2p[i]+'與'+str(j)+'數'+dictionary_a2p[j]+'：'+str(data_p2p[i][j]))
    
    return data_a2p , data_p2p

In [41]:
#好題目評分
def standard( atp1, atp2, atp3, p1tp2, p1tp3, p2tp3):
    sum_a2p = atp1 + atp2 + atp3
    sum_p2p = p1tp2 + p1tp3 + p2tp3
    standard_ = sum_a2p - sum_p2p
    return standard_

In [51]:
#整理題目分數大小，並輸出
def datasort(ans_list,data,name,data_a2p,data_p2p): #data須為dict類別,依照values排列順序
    print(data)
    data.sort(key=lambda x: x[3], reverse=True)
    wp = open("./standard/sum/ptp權重x1/"+name+".txt","w",encoding="utf-8")
    wp.write('atp1, atp2, atp3, p1tp2, p1tp3, p2tp3\n')
    for num in range(0,len(data)):
        wp.write(ans_list[data[num][0]]+" "+ans_list[data[num][1]]+" "+ans_list[data[num][2]]+" "+str(round(data[num][3],3))+" "+str(round(data_a2p[data[num][0]],3))+" "+str(round(data_a2p[data[num][1]],3))+" "+str(round(data_a2p[data[num][2]],3))+" "+str(round(data_p2p[data[num][0]][data[num][1]],3))+" "+str(round(data_p2p[data[num][0]][data[num][2]],3))+" "+str(round(data_p2p[data[num][1]][data[num][2]],3))+"\n")
    wp.close

In [53]:
#找該題目的組合
def find_a2p_pair(dictionary_a2p , data_a2p , data_p2p , key):
    
    a2p_pairdata = []
    count_line = 0
    problem_num = 0
    print(key+' 處理中')
    if(not key in AL_CBOW_Data):
        return
    for p1 in range(len(dictionary_a2p)-2):
        if(not dictionary_a2p[p1] in AL_CBOW_Data):
            print(dictionary_a2p[p1]+'out')
            continue
        #
        atp1 = data_a2p[p1]

        for p2 in range(p1+1,len(dictionary_a2p)-1):
            if(not dictionary_a2p[p2] in AL_CBOW_Data):
                print(dictionary_a2p[p2]+'out')
                continue
            #
            atp2 = data_a2p[p2]
            p1tp2 = data_p2p[p1][p2]
            #
            if(atp1 < p1tp2 or atp2 < p1tp2 or atp1 < 0.2 or atp1 > 0.45):
                count_line+=1
                continue

            for p3 in range(p2+1,len(dictionary_a2p)):
                if(not dictionary_a2p[p3] in AL_CBOW_Data):
                    print(dictionary_a2p[p3]+'out')
                    continue
                #
                atp3 = data_a2p[p3]
                p1tp3 = data_p2p[p1][p3]
                p2tp3 = data_p2p[p2][p3]
                #
                if(atp1 < p1tp3 or atp3 < p1tp3 or atp2 < p2tp3 or atp3 < p2tp3 or atp2 < 0.2 or atp2 > 0.45 or atp3 < 0.2 or atp3 > 0.45):
                    count_line+=1
                    continue

                standard_all = standard( atp1, atp2, atp3, p1tp2, p1tp3, p2tp3)
                a2p_pairdata.append ((p1 , p2 , p3 , standard_all))

                #顯示進度
                problem_num += 1
                count_line+=1
                if(count_line%1000 == 0):
                    clear_output(wait=True)
                    print(key+' 處理中 '+str(count_line))
                    print('共有'+str(problem_num)+'題\n')
                    print(str(p1)+" "+str(p2)+" "+str(p3))
                #顯示完畢

    clear_output(wait=True)
    print(key +' 整理！'+str(problem_num)+'/'+str(count_line))
    datasort(dictionary_a2p , a2p_pairdata , key, data_a2p, data_p2p)
    print('完成：'+key)
                    
    return
                    

In [44]:
#主程式之，找所有題目的題組
def frequency_database_a2p_all():
    pattern = '([\p{Han}]+)' #抓漢字的正則運算法
    
    fp = open("../詞頻聯想/修正2頻3心像表格.txt", "r" ,encoding="utf-8",errors="ignore")
    
    print('載入...')
    line = fp.readline()
    line = fp.readline()
    while(line):
        dictionary_a2p = []
        key = regex.findall(pattern, line) #抓詞頻答案
        line = fp.readline()
        line = fp.readline()
        line = fp.readline()
        ans = regex.findall(pattern, line) #抓詞頻題組
        dictionary_a2p = ans[2:]

        clear_output(wait=True)
        print('算距離階段')
        data_a2p , data_p2p = ready_for_distance(dictionary_a2p,key[0])

        clear_output(wait=True)
        print('找題目階段')
        find_a2p_pair(dictionary_a2p , data_a2p , data_p2p , key[0])

        line = fp.readline()
        clear_output(wait=True)
        print('完成')
    
    fp.close
    return

In [45]:
#先將題庫對應題組做成dictionary
def initial_frequency_database():
    pattern = '([\p{Han}]+)' #抓漢字的正則運算法
    global frequency_database
    frequency_database = {}
    fp = open("../詞頻聯想/修正2頻3心像表格.txt", "r" ,encoding="utf-8",errors="ignore")
    
    print('載入...')
    line = fp.readline()
    line = fp.readline()
    
    while(line):
        key = regex.findall(pattern, line) #抓詞頻答案
        line = fp.readline()
        line = fp.readline()
        line = fp.readline()
        ans = regex.findall(pattern, line) #抓詞頻題組
        frequency_database[key[0]] = ans[2:]
        
        line = fp.readline()
    
    clear_output(wait=True)
    print('完成')
    fp.close

In [46]:
#主程式之二：指定特定詞來做題組
def assign_frequency_database(key):
    if not key in AL_CBOW_Data:
        print('無此字')
        return
    dictionary_a2p = frequency_database[key]
    clear_output(wait=True)
    print('算距離階段')
    data_a2p , data_p2p = ready_for_distance(dictionary_a2p,key)

    clear_output(wait=True)
    print('找題目階段')
    find_a2p_pair(dictionary_a2p , data_a2p , data_p2p , key)

    clear_output(wait=True)
    print('完成')
    
    return

In [12]:
#load 向量集
data = load_vectors('cc.zh.300.vec')

完成


In [29]:
#主程式之，找所有題目的題組
frequency_database_a2p()

計算第7數防禦


KeyboardInterrupt: 

In [19]:
#初始化題庫
initial_frequency_database()

完成


In [47]:
#主程式之二：指定特定詞來做題組
assign_frequency_database('月亮')

完成


In [33]:
cosine_meansure('月亮','耳朵')

0.3492052215166766

In [48]:
assign_frequency_database('瀑布')


完成


In [52]:
assign_frequency_database('車禍')

完成
